In [2]:
# ------------------------------------------
# CELDA 1: Importar bibliotecas necesarias
# ------------------------------------------

import sqlite3  # Importamos la biblioteca para conectarnos a una base de datos SQLite
import csv      # Importamos csv para poder leer archivos CSV
import os       # Importamos os para manejar rutas y archivos en el sistema operativo

# Definimos las rutas de archivos en variables para facilidad de mantenimiento
# Ajusta estas rutas si tienes una estructura de carpetas diferente.
data_folder = "../data"        # Carpeta donde se ubican los archivos CSV
db_folder   = "../db"          # Carpeta donde se guardará la base de datos
db_name     = "production.db"  # Nombre del archivo de base de datos
print(f'Bibliotecas instaladas')

Bibliotecas instaladas
